In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
pd.set_option('display.max_rows', None)

In [43]:
data = pd.read_csv('Real_State_Bogota.csv')
data.head()

,longitude,latitude,stratum,bathrooms,constructed_area,house_age,bedrooms,parking,administration,floor,rent_price,facilities
0,-74.053660,4.660556,6,4,237.0,23,3,3,2600000.0,7,14000000,"['Acceso Pavimentado', 'Alarma', 'Amoblado', '..."
1,-74.046431,4.679071,5,1,28.0,4,1,0,0.0,3,2200000,"['Acceso Pavimentado', 'Ascensor', 'Baño Indep..."
2,-74.048398,4.700991,5,2,97.0,12,2,2,0.0,0,4500000,[]
3,-74.045721,4.671554,6,4,167.0,23,3,2,0.0,3,11900000,"['Ascensor', 'Barra estilo americano', 'Baño A..."
4,-74.050727,4.700429,5,2,78.0,23,3,1,0.0,3,4000000,"['Acceso Pavimentado', 'Ascensor', 'Baño Indep..."


In [44]:
data['facilities'] = data['facilities'].apply(ast.literal_eval)

In [45]:
#Se contabilizan las comodidades mas comunes en data set 
facilities = {}
for i in data['facilities']:
    for j in i:
        facilities[j] = facilities.get(j, 0) + 1

n_rows = data.shape[0]
#Ahora se identificaran las comodidades menos comunes para descartarlas de los datos
low_importance_facilities = []
for i in facilities:
    if (facilities[i]/n_rows) <= 0.05:
        low_importance_facilities.append(i)   

In [49]:
data['facilities'] = data['facilities'].apply(lambda lista: [item for item in lista if item not in low_importance_facilities ])